In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,AveragePooling2D,Flatten
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD,Adam

from tensorflow.keras.layers import LeakyReLU,BatchNormalization

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [2]:
(X_train,Y_train),(X_test,Y_test)=cifar100.load_data()

X_train,X_test=X_train/255,X_test/255
y_train,y_test=to_categorical(Y_train),to_categorical(Y_test)

169009152/169001437 [==============================] - 3s 0us/step


In [3]:
aug_data=ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.3)
aug_data.fit(X_train)

In [4]:
shape=(32,32,3)

inputs = keras.Input(shape=shape)

a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(inputs)  #1
a=layers.BatchNormalization()(a)
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)   #2
a=layers.BatchNormalization()(a)
a_temp=a
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)#\3
a=layers.BatchNormalization()(a)
a=layers.Conv2D(32,(3,3),strides=1,padding="same")(a)#
a=layers.BatchNormalization()(a)

a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)


b=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #1
                               
#a_temp=a
##########################################################################
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #5
a=layers.BatchNormalization()(a)

a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #6
a=layers.BatchNormalization()(a)

a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)

a_temp=a
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #7
a=layers.BatchNormalization()(a)

a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #8
a=layers.BatchNormalization()(a)

a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)


b=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #2

###############################################################################

a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #9
a=layers.BatchNormalization()(a)

a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #10
a=layers.BatchNormalization()(a)

a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)

a_temp=a
a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #11
a=layers.BatchNormalization()(a)

a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #12
a=layers.BatchNormalization()(a)

a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)


b=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #3

#################################################################################

a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #
a=layers.BatchNormalization()(a)

a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #14
a=layers.BatchNormalization()(a)

a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)

a_temp=a
a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #15
a=layers.BatchNormalization()(a)

a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #16
a=layers.BatchNormalization()(a)

a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)

############################################################3
a=layers.AveragePooling2D(pool_size=(2,2),strides=1,padding="same")(a)
a=layers.BatchNormalization()(a)

a=Flatten()(a)

#############################################################

a=layers.Dense(512,activation="relu")(a)
a=layers.BatchNormalization()(a)

outputs=layers.Dense(y_train.shape[1],activation="softmax")(a)


model = keras.Model(inputs, outputs)

In [6]:
adam= Adam(learning_rate=0.0001,clipvalue=0.6)


model.compile(loss="categorical_crossentropy",optimizer=adam,metrics=["accuracy"])
call=EarlyStopping(monitor="val_loss",verbose=1,mode="auto",patience=5,restore_best_weights=True)
checkpoint=ModelCheckpoint('ADAM_WITH_BATCHNORM_RESNET.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True,save_weights_only=True,model='auto',period=1)


In [7]:
model.fit_generator(aug_data.flow(X_train,y_train,batch_size=128),steps_per_epoch=len(X_train)/128,epochs=80,validation_data=(X_test,y_test),verbose=1,callbacks=[call,checkpoint])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/80
391/390 [==============================] - ETA: 0s - loss: 3.5620 - accuracy: 0.1801
Epoch 00001: val_accuracy improved from -inf to 0.05250, saving model to ADAM_WITH_BATCHNORM_RESNET.hdf5
391/390 [==============================] - 224s 573ms/step - loss: 3.5620 - accuracy: 0.1801 - val_loss: 4.6471 - val_accuracy: 0.0525
Epoch 2/80
391/390 [==============================] - ETA: 0s - loss: 2.9344 - accuracy: 0.2821
Epoch 00002: val_accuracy improved from 0.05250 to 0.31100, saving model to ADAM_WITH_BATCHNORM_RESNET.hdf5
391/390 [==============================] - 223s 571ms/step - loss: 2.9344 - accuracy: 0.2821 - val_loss: 2.8649 - val_accuracy: 0.3110
Epoch 3/80
391/390 [==============================] - ETA: 0s - loss: 2.6201 - accuracy: 0.3446
Epoch 00003: val_accuracy improved from 0.31100 to 0.34180, saving model to ADAM_WITH_BATCHNORM_RESNET.hdf5
391/390 [==============================] - 22

In [8]:
y_true = y_test.argmax(-1)
y_pred = model.predict(X_test).argmax(-1)

In [9]:
from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print(accuracy_score(Y_test,y_pred))
print(precision_score(Y_test,y_pred,average="weighted"))
print(recall_score(Y_test,y_pred,average="weighted"))

              precision    recall  f1-score   support

           0       0.83      0.78      0.80       100
           1       0.70      0.68      0.69       100
           2       0.48      0.48      0.48       100
           3       0.48      0.34      0.40       100
           4       0.57      0.36      0.44       100
           5       0.40      0.68      0.51       100
           6       0.63      0.73      0.68       100
           7       0.70      0.59      0.64       100
           8       0.55      0.73      0.63       100
           9       0.78      0.63      0.70       100
          10       0.49      0.35      0.41       100
          11       0.45      0.42      0.44       100
          12       0.62      0.70      0.66       100
          13       0.41      0.60      0.49       100
          14       0.53      0.57      0.55       100
          15       0.42      0.65      0.51       100
          16       0.73      0.53      0.61       100
          17       0.61    

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,AveragePooling2D,Flatten
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD,Adam

from tensorflow.keras.layers import LeakyReLU,BatchNormalization

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

(X_train,Y_train),(X_test,Y_test)=cifar100.load_data()

X_train,X_test=X_train/255,X_test/255
y_train,y_test=to_categorical(Y_train),to_categorical(Y_test)

shape=(32,32,3)

inputs = keras.Input(shape=shape)

a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(inputs)  #1
a=layers.BatchNormalization()(a)
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)   #2
a=layers.BatchNormalization()(a)
a_temp=a
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)#\3
a=layers.BatchNormalization()(a)
a=layers.Conv2D(32,(3,3),strides=1,padding="same")(a)#
a=layers.BatchNormalization()(a)

a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)


b=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #1
                               
#a_temp=a
##########################################################################
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #5
a=layers.BatchNormalization()(a)

a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #6
a=layers.BatchNormalization()(a)

a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)

a_temp=a
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #7
a=layers.BatchNormalization()(a)

a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #8
a=layers.BatchNormalization()(a)

a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)


b=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #2

###############################################################################

a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #9
a=layers.BatchNormalization()(a)

a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #10
a=layers.BatchNormalization()(a)

a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)

a_temp=a
a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #11
a=layers.BatchNormalization()(a)

a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #12
a=layers.BatchNormalization()(a)

a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)


b=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #3

#################################################################################

a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #
a=layers.BatchNormalization()(a)

a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #14
a=layers.BatchNormalization()(a)

a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)

a_temp=a
a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #15
a=layers.BatchNormalization()(a)

a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #16
a=layers.BatchNormalization()(a)

a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
#a=layers.BatchNormalization()(a)

############################################################3
a=layers.AveragePooling2D(pool_size=(2,2),strides=1,padding="same")(a)
a=layers.BatchNormalization()(a)

a=Flatten()(a)

#############################################################

a=layers.Dense(512,activation="relu")(a)
a=layers.BatchNormalization()(a)

outputs=layers.Dense(y_train.shape[1],activation="softmax")(a)


model = keras.Model(inputs, outputs)

model.load_weights('../weights/ADAM_WITH_BATCHNORM_RESNET.hdf5')

y_true = y_test.argmax(-1)
y_pred = model.predict(X_test).argmax(-1)

from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("the accuracy obtained is {}".format(accuracy_score(Y_test,y_pred)))
print("precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("the recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))